<a href="https://colab.research.google.com/github/ipeirotis-org/academiabot/blob/master/01-Pandas/A2-Introduction_Basic_Data_Manipulation_Techniques.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# A2: Basic Data Manipulation — SQL Operations in Pandas

## Building on Your SQL Knowledge

In Notebook A1, you learned to load data and explore it. Now you'll learn to **manipulate** data in Pandas using operations that mirror SQL.

**Why learn these in Pandas if you already know SQL?**

1. Sometimes the data is already in Pandas (from a CSV, API, or prior analysis)
2. You may need to manipulate data between SQL queries
3. Pandas operations can be chained for complex transformations
4. Some operations are easier or more flexible in Pandas

## SQL to Pandas Reference

Here's your cheat sheet for this notebook:

| SQL | Pandas | Purpose |
|-----|--------|--------|
| `SELECT col1, col2` | `.filter(items=[...])` | Select specific columns |
| `SELECT * ... AS alias` | `.rename(columns={...})` | Rename columns |
| `WHERE condition` | `.query('...')` | Filter rows |
| `ORDER BY col` | `.sort_values('col')` | Sort rows |
| `SELECT DISTINCT` | `.drop_duplicates()` | Remove duplicate rows |
| `JOIN` | `pd.merge(...)` | Combine tables |
| `GROUP BY ... AGG()` | `.groupby().agg()` | Aggregate by groups |
| `LIMIT n` | `.head(n)` | Take first n rows |

## What You Will Learn

1. **Select columns** with `.filter()`
2. **Rename columns** with `.rename()`
3. **Filter rows** with `.query()`
4. **Sort data** with `.sort_values()`
5. **Remove duplicates** with `.drop_duplicates()`
6. **Join tables** with `pd.merge()`
7. **Aggregate data** with `.groupby()` and `.agg()`
8. **Chain operations** for concise, readable code

---

## Setup

Run these cells to set up your environment and load the data.

### 🗳️ Quick Check-in

Before we start, let's connect this to what you already know:

- **Show of hands**: How many of you used Notebook A1 (Introduction to Pandas)?
- **Quick quiz**: What's the Pandas equivalent of `SELECT * FROM table LIMIT 5`?
  - Answer: `.head(5)`

> 💡 **Today's goal**: Learn to do in Pandas everything you can do in SQL!

In [ ]:
# Authentication
from google.colab import auth
auth.authenticate_user()
print("✓ Authenticated")

In [ ]:
# BigQuery setup
!pip install -q google-cloud-bigquery
from google.cloud import bigquery

PROJECT_ID = "nyu-datasets"  # ← Change if needed
client = bigquery.Client(project=PROJECT_ID)
print(f"✓ Connected to BigQuery")

In [ ]:
# Import libraries and configure plotting
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

%config InlineBackend.figure_format = 'retina'
plt.rcParams['figure.figsize'] = [10, 4]
sns.set_style("whitegrid")
print("✓ Libraries imported")

In [ ]:
# Load restaurant data
sql = """
SELECT CAMIS, DBA, BUILDING, STREET, ZIPCODE, BORO,
       CUISINE_DESCRIPTION, LATITUDE, LONGITUDE, NTA
FROM `nyu-datasets.doh_restaurants.restaurants`
"""
restaurants = client.query(sql).to_dataframe()
print(f"✓ Loaded {len(restaurants):,} restaurants")

# Load inspection data
sql = """
SELECT R.CAMIS, R.DBA, R.ZIPCODE, R.BORO, R.CUISINE_DESCRIPTION, R.NTA,
       I.INSPECTION_DATE, I.INSPECTION_ID, I.INSPECTION_TYPE, I.SCORE, I.GRADE
FROM `nyu-datasets.doh_restaurants.restaurants` R
JOIN `nyu-datasets.doh_restaurants.inspections` I ON I.CAMIS = R.CAMIS
"""
inspections = client.query(sql).to_dataframe()
print(f"✓ Loaded {len(inspections):,} inspections")

# Load violations from most recent inspection
sql = """
WITH latest_inspection AS (
    SELECT CAMIS, MAX(INSPECTION_DATE) AS INSPECTION_DATE
    FROM `nyu-datasets.doh_restaurants.inspections`
    GROUP BY CAMIS
)
SELECT R.CAMIS, R.DBA, R.ZIPCODE, R.BORO,
       I.INSPECTION_DATE, I.INSPECTION_ID, I.SCORE, I.GRADE,
       V.VIOLATION_CODE
FROM `nyu-datasets.doh_restaurants.restaurants` R
JOIN latest_inspection L ON R.CAMIS = L.CAMIS
JOIN `nyu-datasets.doh_restaurants.inspections` I
     ON I.CAMIS = L.CAMIS AND L.INSPECTION_DATE = I.INSPECTION_DATE
JOIN `nyu-datasets.doh_restaurants.violations` V ON I.INSPECTION_ID = V.INSPECTION_ID
"""
violations = client.query(sql).to_dataframe()
print(f"✓ Loaded {len(violations):,} violations")

---

## Part 1: Selecting Columns — `.filter()`

**SQL equivalent**: `SELECT col1, col2, col3 FROM table`

In Pandas, use `.filter(items=[...])` to select specific columns.

> **Note**: The name `.filter()` can be confusing! In Pandas, `filter` selects *columns*, not rows. To filter *rows*, use `.query()` (covered later).

In [ ]:
# View all columns in inspections
inspections.head(3)

In [ ]:
# Select only specific columns
# SQL: SELECT DBA, GRADE, INSPECTION_DATE FROM inspections

inspections.filter(items=["DBA", "GRADE", "INSPECTION_DATE"])

In [ ]:
# Alternative: use double brackets
# This is equivalent but .filter() is more explicit

inspections[["DBA", "GRADE", "INSPECTION_DATE"]]

### 🎯 Activity 1: Column Selection Practice

**Task 1**: From the `restaurants` DataFrame, select only: `DBA`, `CUISINE_DESCRIPTION`, `BORO`, and `ZIPCODE`

**Task 2**: From the `violations` DataFrame, select: `DBA`, `VIOLATION_CODE`, `INSPECTION_DATE`

**Bonus**: Try both methods — `.filter(items=[...])` and double brackets `[[...]]`

In [ ]:
# Activity 1: Select columns

# Task 1: Select from restaurants


# Task 2: Select from violations


# Bonus: Try double bracket notation



---

## Part 2: Renaming Columns — `.rename()`

**SQL equivalent**: `SELECT col1 AS new_name FROM table`

Use `.rename(columns={...})` with a dictionary mapping old names to new names.

In [ ]:
# Rename columns to be more descriptive
# SQL: SELECT CAMIS AS RESTAURANT_ID, DBA AS NAME, BORO AS BOROUGH FROM restaurants

restaurants.rename(
    columns={
        "CAMIS": "RESTAURANT_ID",
        "DBA": "NAME",
        "BORO": "BOROUGH",
        "BUILDING": "STREET_NUMBER"
    }
).head()

> **Important**: `.rename()` returns a *new* DataFrame. The original `restaurants` DataFrame is unchanged unless you reassign it:
> ```python
> restaurants = restaurants.rename(columns={...})
> ```

### 🎯 Activity 2: Make Columns Human-Readable

The column names in our data are all UPPERCASE and use abbreviations. Let's fix that!

**Task**: Rename columns in the `inspections` DataFrame:
- `DBA` → `restaurant_name`
- `BORO` → `borough`
- `SCORE` → `inspection_score`
- `GRADE` → `letter_grade`

**Question**: After running `.rename()`, is the original DataFrame changed? How would you make it permanent?

In [ ]:
# Activity 2: Rename columns

# Your renaming code here:


# Check: Is the original changed?
# inspections.columns  # What do you see?


---

## Part 3: Filtering Rows — `.query()`

**SQL equivalent**: `SELECT * FROM table WHERE condition`

Use `.query('condition')` to filter rows. The condition is a string that looks like SQL.

### Basic Comparisons

In [ ]:
# Find all Starbucks locations
# SQL: SELECT * FROM restaurants WHERE DBA = 'STARBUCKS'

restaurants.query(' DBA == "STARBUCKS" ')

In [ ]:
# Find restaurants with high violation scores (score > 30)
# SQL: SELECT * FROM inspections WHERE SCORE > 30

inspections.query(' SCORE > 30 ')

### 🎯 Activity 3: Filter Rows Challenge

Use `.query()` to answer these questions:

**Task 1**: Find all restaurants in Brooklyn

**Task 2**: Find all inspections with a score of exactly 0 (perfect score!)

**Task 3**: Find all Chinese restaurants in Manhattan

**Hint**: For Task 3, you need to combine two conditions with `and`

In [ ]:
# Activity 3: Filter rows

# Task 1: Brooklyn restaurants


# Task 2: Perfect inspections (score = 0)


# Task 3: Chinese restaurants in Manhattan



### Combining Conditions (AND, OR)

In [ ]:
# Find violations with mice (code 04L) in zipcode 10012
# SQL: SELECT * FROM violations WHERE VIOLATION_CODE = '04L' AND ZIPCODE = '10012'

violations.query(' VIOLATION_CODE == "04L" and ZIPCODE == "10012" ')

In [ ]:
# Find restaurants in Manhattan OR Brooklyn
# SQL: SELECT * FROM restaurants WHERE BORO = 'Manhattan' OR BORO = 'Brooklyn'

restaurants.query(' BORO == "Manhattan" or BORO == "Brooklyn" ')

### Using IN for Multiple Values

In [ ]:
# Find restaurants in specific zipcodes (near NYU)
# SQL: SELECT * FROM restaurants WHERE ZIPCODE IN ('10012', '10003', '10014')

restaurants.query(' ZIPCODE in ["10012", "10003", "10014"] ')

### Using Variables in Queries

In [ ]:
# Use @ to reference Python variables inside query strings
target_cuisines = ["Mexican", "Italian", "Chinese"]

restaurants.query(' CUISINE_DESCRIPTION in @target_cuisines ')

### Real Analysis: Which Restaurants Have Mice?

Violation code `04L` means "Evidence of mice or live mice". Let's find out which restaurant chains have the most mice violations.

In [ ]:
# Step 1: Filter to only mice violations
has_mice = violations.query(' VIOLATION_CODE == "04L" ')
print(f"Total mice violations: {len(has_mice):,}")

In [ ]:
# Step 2: Count by restaurant name
mice_by_chain = has_mice["DBA"].value_counts()
mice_by_chain.head(20)

In [ ]:
# Step 3: But wait! Chains with more locations will naturally have more violations.
# Let's normalize by the number of locations.

locations_by_chain = restaurants["DBA"].value_counts()

# Calculate violations per location for top chains
top_chains = locations_by_chain.head(25).index  # Top 25 chains by number of locations

mice_rate = (mice_by_chain / locations_by_chain).loc[top_chains].dropna().sort_values(ascending=False)
mice_rate

In [ ]:
# Visualize it
mice_rate.sort_values().plot(kind='barh', color='brown')
plt.xlabel("Mice Violations per Location")
plt.title("Mice Violation Rate by Restaurant Chain (Top 25 Chains)")

### 🎯 Activity 4: Pest Investigation Near NYU

Time to investigate pest violations near campus!

**Context**: Violation codes for pests:
- `04K`: Live roaches
- `04L`: Evidence of mice or live mice  
- `04M`: Live rats
- `04N`: Filth flies or food/refuse/sewage flies
- `04O`: Other pests

**Tasks**:
1. Find all pest violations (codes 04K, 04L, 04M, 04N, 04O) near NYU (zipcodes 10012, 10003, 10014)
2. Which type of pest violation is most common?
3. Which restaurant names appear most often in pest violations?

**Hints**:
- Use `in` for both violation codes and zipcodes
- Use `.value_counts()` to count occurrences

In [ ]:
# Activity 4: Pest investigation

# Define the pest codes and NYU zipcodes
pest_codes = ["04K", "04L", "04M", "04N", "04O"]
nyu_zipcodes = ["10012", "10003", "10014"]

# Task 1: Find all pest violations near NYU


# Task 2: Most common pest type?


# Task 3: Which restaurants have the most pest violations?



---

## Part 4: Sorting — `.sort_values()`

**SQL equivalent**: `SELECT * FROM table ORDER BY col ASC/DESC`

In [ ]:
# Sort by inspection score (ascending = lowest first)
# SQL: SELECT * FROM inspections ORDER BY SCORE ASC

inspections.sort_values("SCORE").head(10)

In [ ]:
# Sort descending (highest first)
# SQL: SELECT * FROM inspections ORDER BY SCORE DESC

inspections.sort_values("SCORE", ascending=False).head(10)

In [ ]:
# Sort by multiple columns
# SQL: SELECT * FROM inspections ORDER BY INSPECTION_DATE DESC, SCORE ASC

inspections.sort_values(
    ["INSPECTION_DATE", "SCORE"],
    ascending=[False, True]  # Date descending, Score ascending
).head(10)

### 🎯 Activity 5: Sorting Scenarios

Practice different sorting scenarios:

**Task 1**: Find the 10 restaurants with the HIGHEST inspection scores (worst restaurants)

**Task 2**: Sort inspections by borough (A-Z), then by score (lowest first within each borough). Show the first 20 rows.

**Task 3**: Find the 5 most recent inspections that resulted in a 'C' grade

In [ ]:
# Activity 5: Sorting practice

# Task 1: Top 10 worst scores


# Task 2: Sort by borough then score


# Task 3: Most recent C grades



---

## Part 5: Removing Duplicates — `.drop_duplicates()`

**SQL equivalent**: `SELECT DISTINCT col1, col2 FROM table`

In [ ]:
# Get unique cuisine types
# SQL: SELECT DISTINCT CUISINE_DESCRIPTION FROM restaurants

restaurants.filter(items=["CUISINE_DESCRIPTION"]).drop_duplicates()

In [ ]:
# Get unique combinations of borough and cuisine
# SQL: SELECT DISTINCT BORO, CUISINE_DESCRIPTION FROM restaurants

(
    restaurants
    .filter(items=["BORO", "CUISINE_DESCRIPTION"])
    .drop_duplicates()
)

---

## Part 6: Chaining Operations

One of Pandas' superpowers is **method chaining**—combining multiple operations in a single, readable statement.

### The Pattern

Wrap your operations in parentheses and put each step on its own line:

```python
(
    df
    .operation1()
    .operation2()
    .operation3()
)
```

In [ ]:
# Find Coffee/Tea shops in 10012, showing just name and address, sorted alphabetically

# SQL equivalent:
# SELECT DISTINCT DBA, BUILDING, STREET
# FROM restaurants
# WHERE CUISINE_DESCRIPTION = 'Coffee/Tea' AND ZIPCODE = '10012'
# ORDER BY DBA ASC

(
    restaurants
    .query(' CUISINE_DESCRIPTION == "Coffee/Tea" and ZIPCODE == "10012" ')
    .filter(items=["DBA", "BUILDING", "STREET"])
    .drop_duplicates()
    .sort_values("DBA")
)

In [ ]:
# Find the 15 most recent mice violations near NYU

(
    violations
    .query(' VIOLATION_CODE == "04L" ')
    .query(' ZIPCODE in ["10012", "10003", "10014"] ')
    .filter(items=["DBA", "INSPECTION_DATE", "ZIPCODE"])
    .drop_duplicates()
    .sort_values("INSPECTION_DATE", ascending=False)
    .head(15)
)

### 🎯 Activity 6: Chain Challenge

Combine everything you've learned in a single chained operation!

**Challenge**: Find Coffee/Tea shops in Manhattan with Grade A, showing only the name and address, sorted alphabetically, with no duplicates.

**SQL equivalent**:
```sql
SELECT DISTINCT DBA, BUILDING, STREET
FROM restaurants R
JOIN inspections I ON R.CAMIS = I.CAMIS
WHERE CUISINE_DESCRIPTION = 'Coffee/Tea'
  AND BORO = 'Manhattan'
  AND GRADE = 'A'
ORDER BY DBA
```

**Hint**: You'll need to chain: `.query()` → `.filter()` → `.drop_duplicates()` → `.sort_values()`

In [ ]:
# Activity 6: Chain challenge

# Start with inspections (it has both restaurant info and grades)
(
    inspections
    # Add your chain here:

)


---

## Part 7: Joining Tables — `pd.merge()`

**SQL equivalent**: `SELECT * FROM table1 JOIN table2 ON table1.key = table2.key`

The `pd.merge()` function combines two DataFrames based on matching column values.

### Join Types

| Join Type | `how=` | Description |
|-----------|--------|-------------|
| Inner | `'inner'` | Only rows that match in both tables |
| Left | `'left'` | All rows from left table, matching from right |
| Right | `'right'` | All rows from right table, matching from left |
| Outer | `'outer'` | All rows from both tables |

In [ ]:
# First, let's get some neighborhood population data to join
nyc_pop_url = 'https://data.cityofnewyork.us/resource/rnsn-acs2.csv'
nyc_pop = pd.read_csv(nyc_pop_url)

# Clean up column names
nyc_pop.columns = ['BOROUGH', 'FIPS', 'NTA_CODE', 'NTA_NAME',
                   'POP_2000', 'POP_2010', 'POP_DIFF', 'POP_PCT_DIFF']

# Keep only relevant columns
nyc_pop = nyc_pop.filter(items=['NTA_CODE', 'NTA_NAME', 'POP_2010'])
nyc_pop = nyc_pop.dropna()
nyc_pop['POP_2010'] = nyc_pop['POP_2010'].astype(int)

print(f"Loaded {len(nyc_pop)} neighborhoods")
nyc_pop.head()

In [ ]:
# Join inspections with population data
# SQL: SELECT * FROM inspections I JOIN nyc_pop P ON I.NTA = P.NTA_CODE

merged = pd.merge(
    left=inspections,
    right=nyc_pop,
    how='inner',
    left_on='NTA',        # Column in left DataFrame
    right_on='NTA_CODE'   # Column in right DataFrame
)

print(f"Merged dataset: {len(merged):,} rows")
merged.head()

In [ ]:
# Now we can analyze inspection scores by neighborhood population!
merged.plot(
    kind='scatter',
    x='POP_2010',
    y='SCORE',
    alpha=0.05,
    s=5
)
plt.xlabel("Neighborhood Population")
plt.ylabel("Inspection Score")
plt.title("Inspection Score vs Neighborhood Population")

---

## Part 8: Grouping and Aggregating — `.groupby()` and `.agg()`

**SQL equivalent**: `SELECT col, AGG(val) FROM table GROUP BY col`

This is one of the most powerful operations in data analysis.

### Basic Grouping

In [ ]:
# Average inspection score by borough
# SQL: SELECT BORO, AVG(SCORE) FROM inspections GROUP BY BORO

inspections.groupby('BORO')['SCORE'].mean()

In [ ]:
# Count inspections by borough
# SQL: SELECT BORO, COUNT(*) FROM inspections GROUP BY BORO

inspections.groupby('BORO')['CAMIS'].count()

### Multiple Aggregations with `.agg()`

The `.agg()` method lets you compute multiple statistics at once.

In [ ]:
# Multiple aggregations per group
# SQL: SELECT BORO, AVG(SCORE), COUNT(*), COUNT(DISTINCT CAMIS) FROM inspections GROUP BY BORO

(
    inspections
    .groupby('BORO')
    .agg(
        avg_score=('SCORE', 'mean'),           # Average of SCORE
        total_inspections=('CAMIS', 'count'),  # Count of rows
        unique_restaurants=('CAMIS', 'nunique') # Count of unique CAMIS
    )
    .sort_values('avg_score')
)

In [ ]:
# Average score by neighborhood, with sorting
(
    merged
    .groupby('NTA_NAME')
    .agg(
        avg_score=('SCORE', 'mean'),
        num_inspections=('CAMIS', 'count'),
        population=('POP_2010', 'first')  # Just take the first value (they're all the same)
    )
    .query('num_inspections >= 100')  # Only neighborhoods with enough data
    .sort_values('avg_score')
    .head(20)
)

### Grouping by Multiple Columns

In [ ]:
# Average score by borough AND grade
# SQL: SELECT BORO, GRADE, AVG(SCORE) FROM inspections GROUP BY BORO, GRADE

(
    inspections
    .groupby(['BORO', 'GRADE'])
    .agg(avg_score=('SCORE', 'mean'))
    .head(15)
)

### Time-Based Aggregation

When the index is a datetime, you can use `.resample()` to aggregate by time periods.

In [ ]:
# Inspections per month over time
(
    inspections
    .groupby('INSPECTION_DATE')
    .agg(num_inspections=('CAMIS', 'count'))
    .resample('1ME')  # Aggregate to monthly
    .sum()
    .plot(title='Monthly Inspections Over Time')
)

### 🎯 Activity 7: Cuisine Performance Analysis

Let's do a complete analysis of which cuisine types perform best in inspections!

**Tasks**:
1. Calculate for each cuisine type:
   - Average inspection score
   - Total number of inspections
   - Number of unique restaurants

2. Filter to cuisines with at least 500 inspections (for statistical significance)

3. Sort by average score (best = lowest)

4. Show the top 10 best and bottom 10 worst performing cuisines

**Bonus**: Create a bar chart of the top 10 best cuisines

In [ ]:
# Activity 7: Cuisine performance analysis

# Step 1: Group by cuisine and calculate aggregations
cuisine_stats = (
    inspections
    .groupby('CUISINE_DESCRIPTION')
    .agg(
        avg_score = ... , # Add your aggregations here
        num_inspections = ..., # Add your aggregations here
        unique_restaurants = ... # Add your aggregations here
    )
)

# Step 2: Filter to cuisines with 500+ inspections


# Step 3: Sort by average score


# Step 4: Show top 10 best
print("Top 10 BEST cuisines (lowest scores):")


# And bottom 10 worst
print("\nTop 10 WORST cuisines (highest scores):")



### 🎯 Activity 8: Inspection Trends Over Time

Let's analyze how inspections have changed over time!

**Tasks**:
1. Group inspections by date and calculate:
   - Average score per day
   - Number of restaurants inspected per day

2. Filter to days with at least 10 inspections

3. Resample to monthly averages using `.resample('1M').mean()`

4. Create a line plot showing average scores over time

**Discussion**: What patterns do you notice? Any COVID impact visible?

In [ ]:
# Activity 8: Time series analysis

# Group by date and aggregate
daily_stats = (
    inspections
    .groupby('INSPECTION_DATE')
    .agg(
        avg_score=('SCORE', 'mean'),
        num_inspections=('CAMIS', 'count')
    )
)

# Filter and resample to monthly


# Create the plot



### 🎯 Activity 9: Neighborhood Population Analysis

Using the `merged` DataFrame (with population data), analyze the relationship between neighborhood characteristics and restaurant performance.

**Tasks**:
1. Calculate for each neighborhood:
   - Average inspection score
   - Number of restaurants
   - Population (just take the first value)

2. Calculate "restaurants per 1000 residents" for each neighborhood

3. Find the 10 neighborhoods with the most restaurants per capita

4. Create a scatter plot: population vs average score

**Discussion**: Is there a relationship between population density and restaurant quality?

In [ ]:
# Activity 9: Population analysis

# Make sure merged DataFrame exists (run the merge cell first!)

# Task 1: Aggregate by neighborhood
neighborhood_stats = (
    merged
    .groupby('NTA_NAME')
    .agg(
        avg_score=('SCORE', 'mean'),
        num_restaurants=('CAMIS', 'nunique'),
        population=('POP_2010', 'first')
    )
)

# Task 2: Calculate restaurants per 1000 residents


# Task 3: Top 10 by restaurants per capita


# Task 4: Scatter plot



---

## Summary: SQL to Pandas Reference

| SQL | Pandas | Example |
|-----|--------|--------|
| `SELECT col1, col2` | `.filter(items=[...])` | `df.filter(items=['name','score'])` |
| `SELECT ... AS alias` | `.rename(columns={...})` | `df.rename(columns={'DBA':'name'})` |
| `WHERE cond` | `.query('cond')` | `df.query('score > 10')` |
| `WHERE col IN (...)` | `.query('col in [...]')` | `df.query('grade in ["A","B"]')` |
| `ORDER BY col` | `.sort_values('col')` | `df.sort_values('score')` |
| `ORDER BY col DESC` | `.sort_values(..., ascending=False)` | `df.sort_values('score', ascending=False)` |
| `SELECT DISTINCT` | `.drop_duplicates()` | `df.drop_duplicates()` |
| `LIMIT n` | `.head(n)` | `df.head(10)` |
| `JOIN ON` | `pd.merge(..., on=...)` | `pd.merge(df1, df2, on='id')` |
| `GROUP BY ... AGG()` | `.groupby().agg()` | `df.groupby('boro').agg(...)` |

### Method Chaining Pattern

```python
(
    df
    .query('condition')
    .filter(items=['col1', 'col2'])
    .drop_duplicates()
    .sort_values('col1')
    .head(10)
)
```

### What's Next?

In **Notebook A3**, you'll learn advanced techniques:
- Pivot tables for multi-dimensional analysis
- Creating new computed columns
- Applying custom functions to data
- Working with dates and time series

---

# 📝 Activity Solutions

**Note**: Try to complete the activities on your own first! Solutions are here for reference and self-checking.

---

In [ ]:
# =============================================================================
# SOLUTION: Activity 1 - Column Selection Practice
# =============================================================================

# Task 1: Select from restaurants
print("Task 1: Restaurant columns")
display(restaurants.filter(items=["DBA", "CUISINE_DESCRIPTION", "BORO", "ZIPCODE"]).head())

# Task 2: Select from violations
print("\nTask 2: Violation columns")
display(violations.filter(items=["DBA", "VIOLATION_CODE", "INSPECTION_DATE"]).head())

# Bonus: Double bracket notation
print("\nBonus: Double bracket method")
display(restaurants[["DBA", "CUISINE_DESCRIPTION", "BORO", "ZIPCODE"]].head())

In [ ]:
# =============================================================================
# SOLUTION: Activity 2 - Make Columns Human-Readable
# =============================================================================

renamed_inspections = inspections.rename(
    columns={
        "DBA": "restaurant_name",
        "BORO": "borough",
        "SCORE": "inspection_score",
        "GRADE": "letter_grade"
    }
)

print("Renamed columns:")
print(renamed_inspections.columns.tolist())

print("\nOriginal columns (unchanged):")
print(inspections.columns.tolist())

print("\n→ To make permanent: inspections = inspections.rename(columns={...})")

In [ ]:
# =============================================================================
# SOLUTION: Activity 3 - Filter Rows Challenge
# =============================================================================

# Task 1: Brooklyn restaurants
print("Task 1: Brooklyn restaurants")
brooklyn = restaurants.query(' BORO == "Brooklyn" ')
print(f"Found {len(brooklyn):,} restaurants in Brooklyn")

# Task 2: Perfect inspections
print("\nTask 2: Perfect inspections (score = 0)")
perfect = inspections.query(' SCORE == 0 ')
print(f"Found {len(perfect):,} perfect inspections")
display(perfect.head())

# Task 3: Chinese restaurants in Manhattan
print("\nTask 3: Chinese restaurants in Manhattan")
chinese_manhattan = restaurants.query(' CUISINE_DESCRIPTION == "Chinese" and BORO == "Manhattan" ')
print(f"Found {len(chinese_manhattan):,} Chinese restaurants in Manhattan")

In [ ]:
# =============================================================================
# SOLUTION: Activity 4 - Pest Investigation Near NYU
# =============================================================================

pest_codes = ["04K", "04L", "04M", "04N", "04O"]
nyu_zipcodes = ["10012", "10003", "10014"]

# Task 1: Find all pest violations near NYU
pests_near_nyu = violations.query(
    ' VIOLATION_CODE in @pest_codes and ZIPCODE in @nyu_zipcodes '
)
print(f"Task 1: Found {len(pests_near_nyu):,} pest violations near NYU")

# Task 2: Most common pest type
print("\nTask 2: Pest type frequency:")
print(pests_near_nyu["VIOLATION_CODE"].value_counts())
print("\n04L (mice) is most common, followed by 04K (roaches)")

# Task 3: Restaurants with most violations
print("\nTask 3: Restaurants with most pest violations near NYU:")
print(pests_near_nyu["DBA"].value_counts().head(10))

In [ ]:
# =============================================================================
# SOLUTION: Activity 5 - Sorting Scenarios
# =============================================================================

# Task 1: Top 10 worst scores
print("Task 1: 10 restaurants with highest (worst) scores:")
display(inspections.sort_values('SCORE', ascending=False).head(10))

# Task 2: Sort by borough then score
print("\nTask 2: Sorted by borough (A-Z) then score (low to high):")
display(
    inspections
    .sort_values(['BORO', 'SCORE'], ascending=[True, True])
    .head(20)
)

# Task 3: Most recent C grades
print("\nTask 3: 5 most recent C grades:")
display(
    inspections
    .query(' GRADE == "C" ')
    .sort_values('INSPECTION_DATE', ascending=False)
    .head(5)
)

In [ ]:
# =============================================================================
# SOLUTION: Activity 6 - Chain Challenge
# =============================================================================

result = (
    inspections
    .query(' CUISINE_DESCRIPTION == "Coffee/Tea" and BORO == "Manhattan" and GRADE == "A" ')
    .filter(items=["DBA", "BUILDING", "STREET"])
    .drop_duplicates()
    .sort_values("DBA")
)

print(f"Found {len(result)} Grade-A Coffee/Tea shops in Manhattan:")
display(result)

In [ ]:
# =============================================================================
# SOLUTION: Activity 7 - Cuisine Performance Analysis
# =============================================================================

cuisine_stats = (
    inspections
    .groupby('CUISINE_DESCRIPTION')
    .agg(
        avg_score=('SCORE', 'mean'),
        total_inspections=('CAMIS', 'count'),
        unique_restaurants=('CAMIS', 'nunique')
    )
    .query('total_inspections >= 500')
    .sort_values('avg_score')
)

print("Top 10 BEST cuisines (lowest scores):")
display(cuisine_stats.head(10))

print("\nTop 10 WORST cuisines (highest scores):")
display(cuisine_stats.tail(10))

# Bonus: Bar chart
cuisine_stats.head(10)['avg_score'].sort_values().plot(
    kind='barh',
    title='Top 10 Best Performing Cuisines (by Avg Score)',
    xlabel='Average Inspection Score (lower is better)'
)

In [ ]:
# =============================================================================
# SOLUTION: Activity 8 - Inspection Trends Over Time
# =============================================================================

daily_stats = (
    inspections
    .groupby('INSPECTION_DATE')
    .agg(
        avg_score=('SCORE', 'mean'),
        num_inspections=('CAMIS', 'count')
    )
)

# Filter to days with 10+ inspections and resample monthly
monthly_stats = (
    daily_stats
    .query('num_inspections >= 10')
    .resample('1M')
    .mean()
)

# Plot
fig, axes = plt.subplots(2, 1, figsize=(12, 6))

monthly_stats['avg_score'].plot(ax=axes[0], title='Average Inspection Score Over Time')
axes[0].set_ylabel('Average Score')

monthly_stats['num_inspections'].plot(ax=axes[1], title='Average Daily Inspections Over Time')
axes[1].set_ylabel('Inspections per Day')

plt.tight_layout()
print("Notice: COVID-19 impact visible in early 2020 (inspection drop)")

In [ ]:
# =============================================================================
# SOLUTION: Activity 9 - Neighborhood Population Analysis
# =============================================================================

# Aggregate by neighborhood
neighborhood_stats = (
    merged
    .groupby('NTA_NAME')
    .agg(
        avg_score=('SCORE', 'mean'),
        num_restaurants=('CAMIS', 'nunique'),
        population=('POP_2010', 'first')
    )
)

# Calculate restaurants per 1000 residents
neighborhood_stats['restaurants_per_1000'] = (
    neighborhood_stats['num_restaurants'] / neighborhood_stats['population'] * 1000
)

# Top 10 by restaurants per capita
print("Top 10 neighborhoods by restaurants per 1000 residents:")
display(
    neighborhood_stats
    .sort_values('restaurants_per_1000', ascending=False)
    .head(10)
)

# Scatter plot
plt.figure(figsize=(10, 6))
plt.scatter(
    neighborhood_stats['population'],
    neighborhood_stats['avg_score'],
    alpha=0.5
)
plt.xlabel('Population')
plt.ylabel('Average Inspection Score')
plt.title('Neighborhood Population vs Average Restaurant Score')

print("\nConclusion: No strong relationship between population and inspection scores")